# Práctico Clase 2

NOMBRE ALUMNO: **Sebastian Latorre**

Diplomado en Machine Learning Aplicado UC - 2024

**Profesor:** Vicente Domínguez

En este práctico utilizaremos metodos latentes para recomendación:
- **Funk SVD:** factorización matricial incorporando regularizacion y optimizacion con gradient descent.

- **NMF:** Non-negative matrix factorization (los valores de factores latentes se dejan como valores positivos)

Utilizaremos la librería **surprise** (https://surpriselib.com/)

Referencia aqui:
https://surprise.readthedocs.io/en/stable/matrix_factorization.html


## Configuración inicial

In [1]:
# descarga de datasets de train, test e información de items
!gdown 1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
!gdown 1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
!gdown 1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo

Downloading...
From: https://drive.google.com/uc?id=1gmOrtPpZpHJ0HeBwtne-kA8Bll4rFWW7
To: /content/u.item
100% 236k/236k [00:00<00:00, 18.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bnLJUEIRx13k4nxN7x7Fa-3L37rXre73
To: /content/u2.base
100% 1.58M/1.58M [00:00<00:00, 49.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i92TtKsgf_3ffef8EVLH9NNArxvF-cMo
To: /content/u2.test
100% 395k/395k [00:00<00:00, 14.5MB/s]


vemos los nombres de los archivos descargados:

In [2]:
ls

sample_data/  u2.base  u2.test  u.item


instalacion e importacion de librerias:

In [3]:
# instalacion de libreria surprise
!pip3 install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357255 sha256=f04ce1c333769a041f763e83ab3fdb7ab840af60cf96d7f8dc10eee54b8cdfdc
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [4]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import NormalPredictor # random rating prediction
from surprise import SVD, NMF  # matrix factorization methods

from surprise.accuracy import rmse

## Análisis exploratorio de datos

### Datos de entrenamiento:

In [5]:
df_train = pd.read_csv('u2.base',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
df_train.head()

,userid,itemid,rating,timestamp
0,1,3,4,878542960
1,1,4,3,876893119
2,1,5,3,889751712
3,1,6,5,887431973
4,1,7,4,875071561


### Datos de test:

In [6]:
df_test = pd.read_csv('u2.test',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)
df_test.head()

,userid,itemid,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,8,1,875072484
3,1,9,5,878543541
4,1,21,1,878542772


## Convertir dataframe de Pandas a formato surprise

In [7]:
reader = Reader(rating_scale=(1, 5))
data_train = Dataset.load_from_df(df_train[['userid', 'itemid', 'rating']], reader)
data_test = Dataset.load_from_df(df_test[['userid', 'itemid', 'rating']], reader)

# procesar data para libreria surprise
data_train = data_train.build_full_trainset()
data_test = [data_test.df.loc[i].to_list() for i in range(len(data_test.df))]


## Rating Aleatorio
- En surprise: `NormalPredictor`

In [8]:
algo_rndm = NormalPredictor()
algo_rndm.fit(data_train)
predictions = algo_rndm.test(data_test)
RMSE = rmse(predictions)

RMSE: 1.5222


## Prediccion de rating utilizando FunkSVD

In [9]:
funk_svd = SVD(n_factors = 10 , reg_all = 0.02)

funk_svd.fit(data_train)
predictions = funk_svd.test(data_test)
RMSE = rmse(predictions)

RMSE: 0.9364


# Prediccion de rating utilizando NMF

In [10]:
nmf = NMF(n_factors = 10 , reg_pu = 0.02)
nmf.fit(data_train)
predictions = nmf.test(data_test)
RMSE = rmse(predictions)

RMSE: 1.0166


*RESPONDER AQUI LAS SIGUIENTES PREGUNTAS*
1. ¿Cual metodo obtiene menores metricas error en terminos de RMSE? (1 pto)  
2. ¿Cómo se comparan con el baseline random? ¿Qué significa? (1 pto)

**RESPUESTAS**
1. El método que obtiene menores métricas de error en términos de RMSE es Funk SVD con un RMSE de 0.9364.
2. Comparado con el baseline random (NormalPredictor) que tiene un RMSE de 1.5222, Funk SVD (0.9364) y NMF (1.0166) tienen un mejor rendimiento. Esto significa que los métodos de factorización matricial, como Funk SVD y NMF, son más precisos en la predicción de ratings que un modelo que asigna ratings aleatorios.


# Analisis de sensibilidad (5 ptos)
Escoger el **mejor metodo obtenido del ejercicio anterior** y hacer un analisis de sensibilidad modificando:
- Factores latentes (`n_factors`):  10, 50, 100, 200, 300, 400, 500, 1000. Mantener constante el factor de regularización en 0.02.
- Factor de regularización  (`reg_all` en `SVD` y `reg_bu` en `NMF`): 0.02 , 0.002 , 0.0002. Mantener constante el numero de factores en 10.

Manteniendo la configuración anterior.

In [14]:
######## ESCRIBIR CODIGO AQUI PARA SENSIBILIDAD DE FACTORES LATENTES ##################
funk_svd = SVD(n_factors = 10)


In [16]:
######## ESCRIBIR CODIGO AQUI PARA SENSIBILIDAD DE FACTOR DE REGULARIZACION ##################
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Convertir el DataFrame original de pandas a Dataset de surprise
reader = Reader(rating_scale=(1, 5))
data_train = Dataset.load_from_df(df_train[['userid', 'itemid', 'rating']], reader)

n_factors_values = [10, 50, 100, 200, 300, 400, 500, 1000]
rmse_values = []

for n_factors in n_factors_values:
    funk_svd = SVD(n_factors=n_factors, reg_all=0.02)
    results = cross_validate(funk_svd, data_train, measures=['RMSE'], cv=5, verbose=True)
    rmse_values.append(results['test_rmse'].mean())

# Mostrar los resultados
for n_factors, rmse in zip(n_factors_values, rmse_values):
    print(f'n_factors: {n_factors}, RMSE: {rmse}')


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9466  0.9448  0.9362  0.9430  0.9422  0.9426  0.0035  
Fit time          0.57    0.95    0.85    0.48    0.47    0.67    0.20    
Test time         0.22    0.34    0.09    0.09    0.10    0.17    0.10    
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9461  0.9455  0.9455  0.9444  0.9387  0.9440  0.0027  
Fit time          0.70    0.70    0.70    0.70    0.73    0.71    0.01    
Test time         0.19    0.08    0.08    0.20    0.09    0.13    0.05    
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9528  0.9361  0.9443  0.9487  0.9532  0.9470  0.0064  
Fit time          1.41    1.61    1.03    1.03    1.04    1.22    0.24    
Test time         0.21    0.

In [12]:
######## ESCRIBIR CODIGO AQUI PARA SENSIBILIDAD DE FACTOR DE REGULARIZACION ##################
funk_svd = SVD(reg_all = 0.02)


In [19]:
######## ESCRIBIR CODIGO AQUI PARA SENSIBILIDAD DE FACTOR DE REGULARIZACION ##################
reg_values = [0.02, 0.002, 0.0002]
rmse_values_reg = []

for reg in reg_values:
    funk_svd = SVD(n_factors=10, reg_all=reg)
    results = cross_validate(funk_svd, data_train, measures=['RMSE'], cv=5, verbose=True)
    rmse_values_reg.append(results['test_rmse'].mean())

# Mostrar los resultados
for reg, rmse in zip(reg_values, rmse_values_reg):
    print(f'reg_all: {reg}, RMSE: {rmse}')


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9425  0.9455  0.9423  0.9463  0.9383  0.9430  0.0028  
Fit time          0.46    0.49    0.46    0.49    0.49    0.48    0.01    
Test time         0.09    0.16    0.23    0.09    0.11    0.13    0.06    
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9497  0.9465  0.9404  0.9433  0.9427  0.9445  0.0032  
Fit time          0.48    0.47    0.48    0.48    0.46    0.47    0.01    
Test time         0.10    0.10    0.08    0.08    0.19    0.11    0.04    
Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9419  0.9441  0.9502  0.9450  0.9425  0.9447  0.0029  
Fit time          0.70    0.70    0.58    0.48    0.47    0.59    0.10    
Test time         0.17    0.

Comentar los siguientes puntos:
- ¿Cuál es el numero óptimo de factores latentes en términos de RMSE considerando el valor `reg_all` o `reg_pu` por defecto? (1 pto)

```
El número óptimo de factores latentes en términos de RMSE es 10, con un RMSE de
0.9425. Este es el valor más bajo obtenido en las pruebas con diferentes números de factores latentes.
```

- ¿Por qué pasado un cierto numero de factores latentes el desempeño empeora? Comente (1 pto)  

```
Pasado un cierto número de factores latentes, el desempeño empeora debido al
sobreajuste (overfitting). Esto ocurre cuando el modelo empieza a capturar el
ruido en los datos en lugar de los patrones generales. Con más factores
latentes, el modelo se vuelve más complejo y puede ajustar mejor el conjunto
de entrenamiento, pero su capacidad de generalización a nuevos datos disminuye,
resultando en un mayor RMSE en los conjuntos de validación.
```

- ¿Cual es el valor óptimo del factor de regularización considerando el valor `n_factors` por defecto?  (1 pto)


```
El valor óptimo del factor de regularización es 0.02, con un RMSE de reg_all: 0.02, RMSE: 0.9429.
Este es el valor más bajo obtenido en las pruebas con diferentes factores de
regularización.
```

